<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Parse Log, a typical ETL
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Example of creating a ETL job on a log file<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>Perform following</b>:
    <ul>
        <li><b>Extract:</b> Read a csv file into a Spark Dataframe</li>
        <li><b>Transform:</b> Perform transformation to extract information from dataframe</li>
        <li><b>Load:</b> Write the analyzed/transformed dataframe to parquet files</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Basic ETL Job Example").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Basic ETL Job<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Use web log files from the <a href="https://www.sec.gov/dera/data/edgar-log-file-data-set.html" target="_blank">US Securities and Exchange Commission website</a> to do a basic ETL for a day of server activity. <br>Extract the fields of interest and load them into persistent storage.
</span>

<span style="font-family:verdana; font-size:1.0em;">
The Databricks File System (DBFS) is an HDFS-like interface to bulk data stores like Amazon's S3 and Azure's Blob storage service.
Use the header option to specify that the first line of the file is the header.
</span>

In [ ]:
if DATA_BRICKS == False:
    csvfile = "../datasets/log20170630.csv"
else:
    csvfile = "dbfs:/FileStore/datasets/log20170630.csv"

In [ ]:
# using a sample to reduce data size
logDF = (spark
         .read
         .option("header", True).csv(csvfile)
         .sample(withReplacement=False, fraction=0.3, seed=2345))

In [ ]:
# display dataframe
showDataFrame(logDF)

## Review the server-side errors, which have error codes in the 500s.

In [ ]:
# col comes from functions library
serverErrorDF = (logDF
                 .filter((col("code") >= 500) & (col("code") < 600))
                 .select("date", "time", "extention", "code"))

In [ ]:
# display dataframe
showDataFrame(serverErrorDF, 10)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Data Validation<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
ETL jobs typically performs validation of data. This includes:<br>
    <ul>
        <li>Look for missing values in any of the fields</li>
        <li>Check if expected fields are present</li>
        <li>Make sure that the expected number of records are present</li>
    </ul>
</span>

In [ ]:
# on the server error dataframe, convert time into GMT format and then take error counts by hour
from pyspark.sql.functions import from_utc_timestamp, hour, col

countsDF = (serverErrorDF
            .select(hour(from_utc_timestamp(col("time"), "GMT")).alias("hour"))
            .groupBy("hour")
            .count()
            .orderBy("hour") )

In [ ]:
# display dataframe
showDataFrame(countsDF)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
For Databricks<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Click on bar chart for the server-side errors by hour to confirm the expectations.<br>
There is an increase in number of errors around mid-night, may be due to server maitenance at that hour
</span>

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Saving Data to databricks file system<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
A common practice includes taking the structured data and writing it back to the file system as parquet file.<br>
<a href="http://parquet.apache.org" target="_blank">Scalable and Optimized Storage - Parquet</a><br>
Databricks file system:<br>
    <ul>
        <li>Use '.write' method on the dataframe to write it in parquet format on file system</li>
        <li>All clusters in databricks can write to '/tmp' which is available for storage. In case of community edition, this space small but is sufficient for most applications</li>
        <li>Use 'dbutils.fs.rm("/tmp/directory", True) to recursively delete items in a directory. This is permanent action</li>
    </ul>
</span>

In [ ]:
if DATA_BRICKS == False:
    workingDir = "../output/"
else:
    workingDir = "FileStore/output/"
targetPath = workingDir + "serverErrorDF.parquet"
print(targetPath)

In [ ]:
# overwrite replaces whatever was there
serverErrorDF.write.mode("overwrite").parquet(targetPath)

In [ ]:
if DATA_BRICKS == False:
    import os
    ret = os.listdir(targetPath)
    for f in ret:
        print(f)
else:
    ret = dbutils.fs.ls("dbfs:/" + targetPath)
    print("Number of files {} Filenames are:".format(len(ret)))
    for f in ret:
        print(f.name)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Summary of ETL pipeline<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
    Remember this is distributed processing which will be useful to scale for large datasets.<br>
    Summary of our ETL pipeline.
</span>

| Code | Stage |
|------|-------|
| **`logDF = (spark.read.option("header", True).csv(<source>))`**                  | **Extract** |
| **`serverErrorDF = (logDF.filter((col("code") >= 500) & (col("code") < 600))`**  | **Transform** |
|                   **`.select("date", "time", "extention", "code"))`**            | **Transform** |
| **`(serverErrorDF.write.parquet(<destination>))`**                               | **Load** |
